In [1]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis as QDA
from sklearn.linear_model import LogisticRegression as LR
from sklearn.neighbors import KernelDensity
from sklearn.model_selection import GridSearchCV

import warnings
warnings.filterwarnings("ignore")

## Ex. 6.12

In [2]:
def get_data(path):
    data=np.loadtxt(path)
    X=data[:, 1:]
    y=data[:, 0]
    return X, y

X_train, y_train=get_data('./data/zip.train')
X_test, y_test=get_data('./data/zip.test')
print(X_train.shape, X_test.shape)
print(np.unique(y_train, return_counts=True)[1])
print(np.unique(y_test, return_counts=True)[1])

(7291, 256) (2007, 256)
[1194 1005  731  658  652  556  664  645  542  644]
[359 264 198 166 200 160 170 147 166 177]


In [3]:
fit1 = LDA().fit(X_train, y_train)
fit2 = QDA().fit(X_train, y_train)
fit3 = LR(random_state=0).fit(X_train, y_train)
result1 = fit1.score(X_train, y_train), fit1.score(X_test, y_test)
result2 = fit2.score(X_train, y_train), fit2.score(X_test, y_test)
result3 = fit3.score(X_train, y_train), fit3.score(X_test, y_test)

print(result1)
print(result2)
print(result3)

(0.9380057605266767, 0.885401096163428)
(0.920175558908243, 0.8156452416542103)
(0.9938280071320806, 0.9108121574489287)


In [4]:
fit0 = LDA(n_components=9).fit(X_train, y_train)
X_train_t = fit0.transform(X_train)
X_test_t = fit0.transform(X_test)

fit1 = LDA().fit(X_train_t, y_train)
fit2 = QDA().fit(X_train_t, y_train)
fit3 = LR(random_state=0).fit(X_train_t, y_train)
result1 = fit1.score(X_train_t, y_train), fit1.score(X_test_t, y_test)
result2 = fit2.score(X_train_t, y_train), fit2.score(X_test_t, y_test)
result3 = fit3.score(X_train_t, y_train), fit3.score(X_test_t, y_test)

print(result1)
print(result2)
print(result3)

(0.9380057605266767, 0.885401096163428)
(0.9502125908654505, 0.8923766816143498)
(0.9478809491153477, 0.8928749377179871)


In [5]:
def fit(X_train, y_train, space):
    nums = []
    kdes = []
    for i in range(10):
        nums.append(np.sum(y_train == i))
        X = X_train[y_train == i]
        grid = GridSearchCV(KernelDensity(), {'bandwidth': space}).fit(X)
        kdes.append(grid.best_estimator_)
        print('{}th ({}) best bandwidth: {}'.format(i+1, nums[-1], kdes[-1].bandwidth))
    
    return nums, kdes

In [ ]:
model = fit(X_train, y_train, np.linspace(0.05, 0.4, 50))

In [ ]:
model = fit(X_train_t, y_train, np.linspace(0.2, 0.9, 50))